In [1]:
from torch.autograd import Variable
import torch
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from scipy.io import loadmat

import glob


%matplotlib inline  



## Part 8

In [2]:
#global variables required for all load_dataset functions
female_actors = ['America Ferrera', 'Angie Harmon', 'Fran Drescher', 'Kristen Chenoweth', 'Lorraine Bracco', 'Peri Gilpin']
male_actors = ['Alec Baldwin', 'Bill Hader', 'Daniel Radcliffe', 'Gerard Butler', 'Michael Vartan', 'Steve Carell']



def scramble(X, Y):
    '''Scrambles two np arrays consistently
     Arguments:
    X -- numpy array
    Y -- numpy array
    
    Returns:
    X[p] -- scrambled version of X
    Y[p] -- scrambled version of Y
    '''
    #reset seed every time so that p is always the same and we get consistent scrambling
    np.random.seed(1) 
    p = np.random.permutation(X.shape[0])
    return X[p], Y[p]

def one_hot_encoder(y, num_class):
    one_hot = []
    for elem in y:
        one_hot.append(np.eye(N = 1, M = num_class, k = int(elem)))
    one_hot = np.array(one_hot)
    return one_hot.reshape((len(one_hot), num_class))

def load_dataset(actors):
    '''Prepare Training, test, and validation data 
    Arguments:
    actors -- list of actors we want to download pictures of 
    training_set_size -- desired training set size
    
    Returns:
    train_x_mat 
    validation_x_mat
    test_x_mat
    train_y
    validation_y
    test_y
    '''
    train_x = []
    train_y = []
    validation_x = []
    validation_y = []
    test_x = []
    test_y = []
    
    class_num = 0
    class_tot = len(actors)
    
    for act in actors:
        #get all files under person's name 
        if act in female_actors:
            files = glob.glob('get_images/cropped/female/' + ''.join(act.split()) + '*')
     
        elif act in male_actors:
            files = glob.glob('get_images/cropped/male/' + ''.join(act.split()) + '*')
        
        
        #check to see if you have enough images for the split

        test_size = 20
        training_size = int((len(files) - test_size) * 0.8)
        validation_size = int((len(files) - test_size) * 0.2)

        for i in range(len(files)):
            
            img = np.array((mpimg.imread(files[i])).flatten() * (1/255.0)) #divide by 255 to normalize
            if i < test_size:
                test_x.append(img)
                test_y.append(class_num)
            
            elif i >= test_size and i < test_size + training_size:
                train_x.append(img)
                train_y.append(class_num)
            else:
                validation_x.append(img)
                validation_y.append(class_num)
                
        class_num += 1
      

    
    train_x_mat = np.vstack(train_x)
    validation_x_mat = np.vstack(validation_x)
    test_x_mat = np.vstack(test_x)

    train_y = np.array(train_y)
    train_y = train_y.reshape((train_y.shape[0], 1))

    validation_y = np.array(validation_y)
    validation_y = validation_y.reshape((validation_y.shape[0], 1))

    test_y = np.array(test_y, ndmin=1)
    test_y = test_y.reshape((test_y.shape[0], 1))

    #scramble data
    train_x_mat, train_y = scramble(train_x_mat, train_y)
    validation_x_mat, validation_y = scramble(validation_x_mat, validation_y)
    test_x_mat,test_y = scramble(test_x_mat,test_y)

    #one hot encoding
    train_y = one_hot_encoder(train_y, class_tot)
    validation_y = one_hot_encoder(validation_y, class_tot)
    test_y = one_hot_encoder(test_y, class_tot)

    return train_x_mat,validation_x_mat,test_x_mat,train_y,validation_y,test_y


In [3]:
test = ['Lorraine Bracco', 'Peri Gilpin', 'Angie Harmon', 'Alec Baldwin', 'Bill Hader', 'Steve Carell']
#train_x, validation_x, test_x, train_y, validation_y, test_y = load_dataset(test)
train_x_mat,validation_x_mat,test_x_mat,train_y,validation_y,test_y = load_dataset(test)
print(train_x_mat.shape)
print(train_y.shape)

print(test_x_mat.shape)
print(test_y.shape)

(363, 1024)
(363, 6)
(120, 1024)
(120, 6)


In [4]:
dim_x = 32*32
dim_h = 100
dim_out = 6

dtype_float = torch.FloatTensor
dtype_long = torch.LongTensor

################################################################################
#Subsample the training set for faster training

x = Variable(torch.from_numpy(train_x_mat), requires_grad=False).type(dtype_float)
y = Variable(torch.from_numpy(train_y), requires_grad=False).type(dtype_long)

#################################################################################


In [5]:
model = torch.nn.Sequential(
    torch.nn.Linear(dim_x, dim_h),
    torch.nn.ReLU(),
    torch.nn.Linear(dim_h, dim_out),
)

In [6]:
loss_fn = torch.nn.CrossEntropyLoss()


In [7]:
import math

def generate_mini_batches(X, Y, mini_batch_size):
    m = X.shape[0]
    X_mini_batch = []
    Y_mini_batch = []
    num_complete_minibatches = math.floor(m/mini_batch_size)
    
    for k in range(0, num_complete_minibatches):
        X_batch = X[k * mini_batch_size:(k + 1) * mini_batch_size,:]
        Y_batch = Y[k * mini_batch_size:(k + 1) * mini_batch_size,:]
        X_mini_batch.append(X_batch)
        Y_mini_batch.append(Y_batch)
    
    if m % mini_batch_size != 0:
        end = (k + 1) * mini_batch_size
        X_batch = X[end:,:]
        Y_batch = Y[end:,:]
        
    return X_mini_batch, Y_mini_batch



In [8]:
mini_X, mini_Y = generate_mini_batches(train_x_mat, train_y, 64)


learning_rate = 1e-2
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for t in range(100):
    for i in range(len(mini_X)):
        batch_x = mini_X[i]
        batch_y = mini_Y[i] #reshape to 1D
        batch_y = np.argmax(batch_y,1)
        
        bx = Variable(torch.from_numpy(batch_x), requires_grad=False).type(dtype_float)
        by = Variable(torch.from_numpy(batch_y), requires_grad=False).type(dtype_long)
        
        y_pred = model(bx)
        loss = loss_fn(y_pred, by)

        optimizer.zero_grad()  # Zero out the previous gradient computation
        loss.backward()    # Compute the gradient
        optimizer.step()   # Use the gradient information to 
                           # make a step
    if (i %1000):
        print(loss)
            
    # print statistics
    
    #if t % 1000 == 0:    
        #print(loss)
    


Variable containing:
 3.1455
[torch.FloatTensor of size 1]

Variable containing:
 1.6670
[torch.FloatTensor of size 1]

Variable containing:
 1.6237
[torch.FloatTensor of size 1]

Variable containing:
 1.4269
[torch.FloatTensor of size 1]

Variable containing:
 1.3598
[torch.FloatTensor of size 1]

Variable containing:
 1.2555
[torch.FloatTensor of size 1]

Variable containing:
 1.1452
[torch.FloatTensor of size 1]

Variable containing:
 1.1011
[torch.FloatTensor of size 1]

Variable containing:
 1.0531
[torch.FloatTensor of size 1]

Variable containing:
 0.9639
[torch.FloatTensor of size 1]

Variable containing:
 0.9010
[torch.FloatTensor of size 1]

Variable containing:
 0.8473
[torch.FloatTensor of size 1]

Variable containing:
 0.7981
[torch.FloatTensor of size 1]

Variable containing:
 0.7724
[torch.FloatTensor of size 1]

Variable containing:
 0.7164
[torch.FloatTensor of size 1]

Variable containing:
 0.6562
[torch.FloatTensor of size 1]

Variable containing:
 0.6164
[torch.Floa

In [9]:
x = Variable(torch.from_numpy(test_x_mat), requires_grad=False).type(dtype_float)
y_pred = model(x).data.numpy()

np.mean(np.argmax(y_pred, 1) == np.argmax(test_y,1))


0.83333333333333337

RuntimeError: multi-target not supported at /Users/soumith/minicondabuild3/conda-bld/pytorch_1512381214802/work/torch/lib/THNN/generic/ClassNLLCriterion.c:22

In [34]:
mini_X, mini_Y = generate_mini_batches(train_x_mat, train_y, 64)


learning_rate = 1e-2
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for t in range(100):

        
    bx = Variable(torch.from_numpy(train_x_mat), requires_grad=False).type(dtype_float)
    by = Variable(torch.from_numpy(np.argmax(train_y,1)), requires_grad=False).type(dtype_long)

    y_pred = model(bx)
    loss = loss_fn(y_pred, by)

    optimizer.zero_grad()  # Zero out the previous gradient computation
    loss.backward()    # Compute the gradient
    optimizer.step()   # Use the gradient information to 
                       # make a step
    if (i %1000):
        print(loss)
            

Variable containing:
 1.7529
[torch.FloatTensor of size 1]

Variable containing:
 1.7528
[torch.FloatTensor of size 1]

Variable containing:
 1.7527
[torch.FloatTensor of size 1]

Variable containing:
 1.7526
[torch.FloatTensor of size 1]

Variable containing:
 1.7526
[torch.FloatTensor of size 1]

Variable containing:
 1.7526
[torch.FloatTensor of size 1]

Variable containing:
 1.7526
[torch.FloatTensor of size 1]

Variable containing:
 1.7526
[torch.FloatTensor of size 1]

Variable containing:
 1.7526
[torch.FloatTensor of size 1]

Variable containing:
 1.7526
[torch.FloatTensor of size 1]

Variable containing:
 1.7526
[torch.FloatTensor of size 1]

Variable containing:
 1.7526
[torch.FloatTensor of size 1]

Variable containing:
 1.7526
[torch.FloatTensor of size 1]

Variable containing:
 1.7526
[torch.FloatTensor of size 1]

Variable containing:
 1.7526
[torch.FloatTensor of size 1]

Variable containing:
 1.7526
[torch.FloatTensor of size 1]

Variable containing:
 1.7526
[torch.Floa

In [35]:
x = Variable(torch.from_numpy(test_x_mat), requires_grad=False).type(dtype_float)
y_pred = model(x).data.numpy()

np.mean(np.argmax(y_pred, 1) == np.argmax(test_y,1))


0.16666666666666666